In [1]:
import gym
import rospy
import numpy as np

from gym.envs.registration import register
from neuroracer_gym.tasks import neuroracer_discrete_task

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

rospy.init_node('neuroracer_qlearn', anonymous=True, log_level=rospy.INFO)


None


In [2]:
env = gym.make('NeuroRacer-v0')

[ERROR] [1609074648.963383, 0.000000]: NOT Initialising Simulation Physics Parameters
[WARN] [1609074648.969471, 0.000000]: Start Init ControllersConnection
[WARN] [1609074648.970839, 0.000000]: END Init ControllersConnection


In [10]:
time_step = env.reset()
print('Time step:')
#print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
#print(next_time_step)

Time step:
Next time step:


In [16]:
print(time_step.shape)
print(len(next_time_step))
print(next_time_step[0].shape)
print(next_time_step[1])
print(next_time_step[2])
print(next_time_step[3])

(12, 90, 3)
4
(12, 90, 3)
6.752982139587402
False
{}


In [5]:
train_env = tf_py_environment.TFPyEnvironment(env)

TypeError: Environment should implement py_environment.PyEnvironment
  In call to configurable 'TFPyEnvironment' (<class 'tf_agents.environments.tf_py_environment.TFPyEnvironment'>)